In [ ]:
!pip install sagemaker --quiet

In [ ]:
import sagemaker
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import TrainingStep, ProcessingStep, CreateModelStep, CacheConfig
from sagemaker.workflow.parameters import ParameterString
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.model_step import ModelStep
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import boto3

sess = sagemaker.Session()

sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

## Manually Deploy a Model from S3

In [ ]:
import json

from sagemaker.huggingface import HuggingFaceModel
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.huggingface import get_huggingface_llm_image_uri

# Model s3 uri 
model_artifact_s3_uri = "s3://sagemaker-us-west-2-539656205201/huggingface-qlora-microsoft-Phi-3-5-min-2024-09-24-19-25-26-124/output/model.tar.gz"

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  session=sess,
)

config = {
    'HF_MODEL_ID': "/opt/ml/model", # path to where sagemaker stores the model
    'SM_NUM_GPUS': json.dumps(1), # Number of GPU used per replica
    'MAX_INPUT_LENGTH': json.dumps(1024), # Max length of input text
    'MAX_TOTAL_TOKENS': json.dumps(2048), # Max length of the generation (including input text)
}

model = HuggingFaceModel( # https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html#hugging-face-model
    model_data=model_artifact_s3_uri,
    image_uri=llm_image,
    sagemaker_session=sess,
    role=role,
    env=config
)


predictor = model.deploy(initial_instance_count=1, instance_type='ml.g5.4xlarge', container_startup_health_check_timeout=1000) # https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy

## Manually Deploy a Model from S3 (uncompressed)

In [ ]:
# from sagemaker.huggingface import HuggingFaceModel
# from sagemaker.workflow.step_collections import RegisterModel
# from sagemaker.huggingface import get_huggingface_llm_image_uri

# # Model s3 prefix(folder) uri  
# model_s3_prefix_uri = "s3://sagemaker-us-west-2-097724924093/huggingface-qlora-microsoft-Phi-3-5-min-2024-09-11-04-03-17-319/output/model/"

# # retrieve the llm image uri
# llm_image = get_huggingface_llm_image_uri(
#   "huggingface",
#   session=sess,
# )

# config = {
#     'HF_MODEL_ID': "/opt/ml/model", # path to where sagemaker stores the model
#     'SM_NUM_GPUS': json.dumps(1), # Number of GPU used per replica
#     'MAX_INPUT_LENGTH': json.dumps(1024), # Max length of input text
#     'MAX_TOTAL_TOKENS': json.dumps(2048), # Max length of the generation (including input text)
# }

# model = HuggingFaceModel( # https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html#hugging-face-model
#     model_data={'S3DataSource':{'S3Uri': model_s3_prefix_uri,'S3DataType': 'S3Prefix','CompressionType': 'None'}}, # We use a dict, for more details, please refer to these two documents: https://sagemaker.readthedocs.io/en/stable/api/inference/model.html, https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_S3ModelDataSource.html
#     role=role,
#     image_uri=llm_image,
#     sagemaker_session=sess,
#     env=config
# )


# predictor = model.deploy(initial_instance_count=1, instance_type='ml.g5.2xlarge', container_startup_health_check_timeout=1000) # https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy